In [2]:
import sionna 
import os
import tensorflow as tf
import numpy as np

In [3]:
from sionna.mapping import Constellation, Mapper, Demapper
from sionna.fec.polar import PolarEncoder, Polar5GEncoder, PolarSCLDecoder, Polar5GDecoder, PolarSCDecoder
from sionna.fec.ldpc import LDPC5GEncoder, LDPC5GDecoder
from sionna.fec.polar.utils import generate_5g_ranking, generate_rm_code
from sionna.fec.conv import ConvEncoder, ViterbiDecoder, BCJRDecoder
from sionna.fec.turbo import TurboEncoder, TurboDecoder
from sionna.fec.linear import OSDecoder
from sionna.utils import BinarySource, ebnodb2no
from sionna.utils.metrics import count_block_errors
from sionna.channel import AWGN
from sionna.utils.plotting import PlotBER
sionna.config.seed = 42

In [4]:
import matplotlib.pyplot as plt

In [5]:
class System_Model(tf.keras.Model):
    def __init__(self, k, n, num_bits_per_symbol, encoder, decoder, demapping_method='app',
                 sim_esno=False, cw_estimates=False):
        super().__init__()
        self.k = k
        self.n = n
        self.sim_esno = sim_esno
        self.cw_estimates = cw_estimates
        self.num_bits_per_symbol = num_bits_per_symbol
        self.source = BinarySource()
        self.constellation = Constellation('qam', num_bits_per_symbol=self.num_bits_per_symbol)
        self.mapper = Mapper(constellation=self.constellation)
        self.demapper = Demapper(demapping_method, constellation=self.constellation)
        self.channel = AWGN()
        self.encoder = encoder
        self.decoder = decoder

    @tf.function()
    def call(self, batch_size, ebno_db):
        if self.sim_esno:
            no = ebnodb2no(ebno_db, num_bits_per_symbol=1, coderate=1)
        else:
            no = ebnodb2no(ebno_db, num_bits_per_symbol=self.num_bits_per_symbol,
                           coderate=self.k/self.n)
        u = self.source([batch_size, self.k])
        c = self.encoder(u)
        x = self.mapper(c)
        y = self.channel([x, no])
        llr_ch = self.demapper([y, no])
        u_hat = self.decoder(llr_ch)
        if self.cw_estimates:
            return c, u_hat
        return u, u_hat

In [6]:
## codes for simulation
k = 64
n = 128
codes_under_test = []
enc = LDPC5GEncoder(k=k, n=n)
dec = LDPC5GDecoder(enc, num_iter=20)
name = '5G LDPC BP-20'
codes_under_test.append([enc, dec, name])

#polar codes
enc = Polar5GEncoder(k=k, n=n)
dec = Polar5GDecoder(enc, dec_type='SC')
name = '5G Polar+ CRC SC'
codes_under_test.append([enc, dec, name])

#polar using SCL 
enc = Polar5GEncoder(k=k, n=n)
dec = Polar5GDecoder(enc, dec_type='SCL', list_size=8)
name = '5G POLAR + CRC SCL-8'
codes_under_test.append([enc, dec, name])


# non 5G schems
#RM codes with SCL
f,_,_,_,_ = generate_rm_code(3, 7)
enc = PolarEncoder(f, n)
dec = PolarSCLDecoder(f,n, list_size=8)
name = "RM read Muller"
codes_under_test.append([enc, dec, name])

#conv viterbi decoding 
enc = ConvEncoder(rate=1/2, constraint_length=8)
dec = ViterbiDecoder(gen_poly=enc.gen_poly, method='soft_llr')
name = "Conv. Code Viterbi (constraint length 8)"
codes_under_test.append([enc, dec, name])

#turbo codes
enc = TurboEncoder(rate=1/2, constraint_length=4, terminate=False)
dec = TurboDecoder(enc, num_iter=8)
name = 'Turbo codes (len 4)'
codes_under_test.append([enc, dec, name])

In [7]:
# to save and plot the BER simulations
ber_plot128 = PlotBER(f'short len code performance (k={k}, n={n})')

In [ ]:
num_bits_per_symbol = 2
ebno_db = np.arange(0,5, 0.5)
for code in codes_under_test:
    print('\nRunning: '+ code[2])
    model = System_Model(k=k, n=n, num_bits_per_symbol=num_bits_per_symbol,
                        encoder=code[0], decoder=code[1])
    ber_plot128.simulate(model,
                         ebno_dbs=ebno_db,
                         legend=code[2],
                         max_mc_iter=100,
                         num_target_block_errors=1000,
                         batch_size=10000,
                         soft_estimates=False,
                         early_stop=True,
                         show_fig=False,
                         add_bler=True,
                         forward_keyboard_interrupt=True)

ber_plot128(ylim=(1e-5, 1), show_bler=False)
    


Running: 5G LDPC BP-20
EbNo [dB] |        BER |       BLER |  bit errors |    num bits | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
      0.0 | 1.6585e-01 | 8.6430e-01 |      106145 |      640000 |         8643 |       10000 |         6.0 |reached target block errors
      0.5 | 1.2742e-01 | 7.1230e-01 |       81549 |      640000 |         7123 |       10000 |         2.3 |reached target block errors
      1.0 | 8.6655e-02 | 5.0330e-01 |       55459 |      640000 |         5033 |       10000 |         2.5 |reached target block errors
      1.5 | 5.0352e-02 | 3.0320e-01 |       32225 |      640000 |         3032 |       10000 |         2.5 |reached target block errors
      2.0 | 2.5684e-02 | 1.5460e-01 |       16438 |      640000 |         1546 |       10000 |         2.3 |reached target block errors
      2.5 | 1.1028e-02 | 6.8500e-02 |    

In [ ]:
ber_plot128(ylim=(1e-5, 1), 